<a href="https://colab.research.google.com/github/Ahmed11Raza/Python-Projects/blob/main/result_management_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# # Result Management System
# A comprehensive system for managing student grades and academic records
\


In [2]:
import json
from typing import Dict, List, Optional

class Student:
    """Class representing a student and their academic records"""
    def __init__(self, student_id: str, name: str):
        self.student_id = student_id
        self.name = name
        self.grades: Dict[str, str] = {}  # {course_id: grade}

    def to_dict(self):
        return {
            'student_id': self.student_id,
            'name': self.name,
            'grades': self.grades
        }

class Course:
    """Class representing academic course information"""
    def __init__(self, course_id: str, name: str, credit: int):
        self.course_id = course_id
        self.name = name
        self.credit = credit

    def to_dict(self):
        return {
            'course_id': self.course_id,
            'name': self.name,
            'credit': self.credit
        }


class ResultManagementSystem:
    """Core system for managing academic results and records"""

    GRADE_POINTS = {'A': 4.0, 'B': 3.0, 'C': 2.0, 'D': 1.0, 'F': 0.0}

    def __init__(self):
        self.students: Dict[str, Student] = {}
        self.courses: Dict[str, Course] = {}

    def add_student(self, student_id: str, name: str) -> bool:
        if student_id in self.students:
            print(f"Student {student_id} already exists.")
            return False
        self.students[student_id] = Student(student_id, name)
        return True

    def add_course(self, course_id: str, name: str, credit: int) -> bool:
        if course_id in self.courses:
            print(f"Course {course_id} already exists.")
            return False
        if credit <= 0:
            print("Credit hours must be positive.")
            return False
        self.courses[course_id] = Course(course_id, name, credit)
        return True

    def record_grade(self, student_id: str, course_id: str, grade: str) -> bool:
        grade = grade.upper()
        if student_id not in self.students:
            print(f"Student {student_id} not found.")
            return False
        if course_id not in self.courses:
            print(f"Course {course_id} not found.")
            return False
        if grade not in self.GRADE_POINTS:
            print(f"Invalid grade {grade}.")
            return False

        student = self.students[student_id]
        student.grades[course_id] = grade
        return True

    def calculate_gpa(self, student_id: str) -> Optional[float]:
        if student_id not in self.students:
            print(f"Student {student_id} not found.")
            return None

        student = self.students[student_id]
        total_points = 0.0
        total_credits = 0

        for course_id, grade in student.grades.items():
            course = self.courses.get(course_id)
            if course:
                total_points += self.GRADE_POINTS[grade] * course.credit
                total_credits += course.credit

        return total_points / total_credits if total_credits else 0.0

    def generate_student_report(self, student_id: str) -> Optional[Dict]:
        if student_id not in self.students:
            return None

        student = self.students[student_id]
        report = {
            'student_id': student_id,
            'name': student.name,
            'grades': [],
            'gpa': self.calculate_gpa(student_id)
        }

        for course_id, grade in student.grades.items():
            course = self.courses.get(course_id)
            if course:
                report['grades'].append({
                    'course': course.name,
                    'grade': grade,
                    'credit': course.credit
                })

        return report

    def generate_course_report(self, course_id: str) -> Optional[Dict]:
        if course_id not in self.courses:
            return None

        course = self.courses[course_id]
        report = {
            'course_id': course_id,
            'name': course.name,
            'enrolled_students': []
        }

        for student in self.students.values():
            if course_id in student.grades:
                report['enrolled_students'].append({
                    'student_id': student.student_id,
                    'name': student.name,
                    'grade': student.grades[course_id]
                })

        return report

    def save_data(self, filename: str = "data.json") -> bool:
        try:
            data = {
                'students': [s.to_dict() for s in self.students.values()],
                'courses': [c.to_dict() for c in self.courses.values()]
            }
            with open(filename, 'w') as f:
                json.dump(data, f)
            return True
        except Exception as e:
            print(f"Error saving data: {e}")
            return False

    def load_data(self, filename: str = "data.json") -> bool:
        try:
            with open(filename, 'r') as f:
                data = json.load(f)

            for course_data in data.get('courses', []):
                self.add_course(
                    course_data['course_id'],
                    course_data['name'],
                    course_data['credit']
                )

            for student_data in data.get('students', []):
                self.add_student(
                    student_data['student_id'],
                    student_data['name']
                )
                student = self.students[student_data['student_id']]
                student.grades = student_data['grades']

            return True
        except FileNotFoundError:
            print("No existing data file found.")
            return False
        except Exception as e:
            print(f"Error loading data: {e}")
            return False


rms = ResultManagementSystem()

rms.load_data()

# Add sample students
rms.add_student("S001", "Alice Johnson")
rms.add_student("S002", "Bob Smith")
rms.add_student("S003", "Charlie Brown")
rms.add_student("S004", "David Lee")

# Add sample courses
rms.add_course("C001", "Mathematics", 3)
rms.add_course("C002", "Physics", 4)
rms.add_course("C003", "Computer Science", 3)
rms.add_course("C004", "History", 2)

# Record grades
rms.record_grade("S001", "C001", "A")
rms.record_grade("S001", "C002", "B")
rms.record_grade("S002", "C001", "C")
rms.record_grade("S002", "C003", "A")
rms.record_grade("S003", "C002", "B")
rms.record_grade("S003", "C003", "A")

# Save data
rms.save_data()

# Generate and display student report
print("Student Report for S001:")
student_report = rms.generate_student_report("S001")
print(json.dumps(student_report, indent=2))
print("Student Report for S002:")
student_report = rms.generate_student_report("S002")
print(json.dumps(student_report, indent=2))
print("Student Report for S003:")
student_report = rms.generate_student_report("S003")
print(json.dumps(student_report, indent=2))

# Generate and display course report
print("\nCourse Report for C001:")
course_report = rms.generate_course_report("C001")
print(json.dumps(course_report, indent=2))
print("Course Report for C002:")
course_report = rms.generate_course_report("C002")
print(json.dumps(course_report, indent=2))
print("Course Report for C003:")
course_report = rms.generate_course_report("C003")
print(json.dumps(course_report, indent=2))


from google.colab import drive
drive.mount('/content/drive')


DRIVE_PATH = "/content/drive/MyDrive/Colab Data/"



rms.save_data(DRIVE_PATH + "student_data.json")
rms.save_data(DRIVE_PATH + "course_data.json")
rms.load_data(DRIVE_PATH + "student_data.json")
rms.load_data(DRIVE_PATH + "course_data.json")





Student S001 already exists.
Student S002 already exists.
Student S003 already exists.
Student S004 already exists.
Course C001 already exists.
Course C002 already exists.
Course C003 already exists.
Course C004 already exists.
Student Report for S001:
{
  "student_id": "S001",
  "name": "Alice Johnson",
  "grades": [
    {
      "course": "Mathematics",
      "grade": "A",
      "credit": 3
    },
    {
      "course": "Physics",
      "grade": "B",
      "credit": 4
    }
  ],
  "gpa": 3.4285714285714284
}
Student Report for S002:
{
  "student_id": "S002",
  "name": "Bob Smith",
  "grades": [
    {
      "course": "Mathematics",
      "grade": "C",
      "credit": 3
    },
    {
      "course": "Computer Science",
      "grade": "A",
      "credit": 3
    }
  ],
  "gpa": 3.0
}
Student Report for S003:
{
  "student_id": "S003",
  "name": "Charlie Brown",
  "grades": [
    {
      "course": "Physics",
      "grade": "B",
      "credit": 4
    },
    {
      "course": "Computer Science"

False